# Random Forest Classification

In [1]:
import pandas as pd

## Random Forest

What is a random forest?
How can we use it?
Why should we use it?
TODO

In [2]:
# Getting the data from the urls
url_red_wine = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
url_white_wine = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv'

red_wine_data = pd.read_csv(url_red_wine, sep = ';');
white_wine_data = pd.read_csv(url_white_wine, sep = ';');

In [3]:
# Adding the good quality column to the data
red_wine_data["good quality"] = red_wine_data.apply(lambda x: x["quality"] >= 7, axis=1)
white_wine_data["good quality"] = white_wine_data.apply(lambda x: x["quality"] >= 7, axis=1)

In [4]:
# Shuffle the data
red_wine_data = red_wine_data.sample(frac = 1).reset_index(drop = True)
white_wine_data = white_wine_data.sample(frac = 1).reset_index(drop = True)

# Splitting the data into training and testing data
train_amount_red = int(0.75 * len(red_wine_data))
train_red_wine = red_wine_data.head(train_amount_red)
test_red_wine = red_wine_data.tail(len(red_wine_data) - train_amount_red)

train_amount_white = int(0.75 * len(white_wine_data))
train_white_wine = white_wine_data.head(train_amount_white)
test_white_wine = white_wine_data.tail(len(white_wine_data) - train_amount_white)

# Splitting the data into measurements and labels
train_red_wine_measurements = train_red_wine.drop(["quality", "good quality"], axis = 1).values
train_red_wine_labels = train_red_wine["good quality"]

test_red_wine_measurements = test_red_wine.drop(["quality", "good quality"], axis = 1).values
test_red_wine_labels = test_red_wine["good quality"]

train_white_wine_measurements = train_white_wine.drop(["quality", "good quality"], axis = 1).values
train_white_wine_labels = train_white_wine["good quality"]

test_white_wine_measurements = test_white_wine.drop(["quality", "good quality"], axis = 1).values
test_white_wine_labels = test_white_wine["good quality"]

In [ ]:
# Random Forest Classifier
# TODO